In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *

my_env = 'data'
my_key = get_key('koray_data')

In [ ]:
# find all runs from 4dn lab without a contributing lab
q = "/search/?lab.display_title=4DN%20DCIC%20Lab%2C%20HMS&type=FileProcessed"
all_awsem_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_awsem_resp))
files_with_source = [i for i in all_awsem_resp if i.get('source_experiments')]
print(len(files_with_source))
## Patching all processed files
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
def get_attr_from_pc_file(file_resp):
    lab = ''
    award = ''
    source_exp = file_resp['source_experiments'][0]
    exp_info = ff_utils.get_metadata(source_exp, key=my_key, add_on='frame=raw')
    lab = exp_info['lab']
    award = exp_info['award']
    return lab,award

counter = 0
for a_file in files_with_source:
    counter += 1
    if counter % 100 == 0:
        print(counter)
    my_lab, my_award = get_attr_from_pc_file(a_file)
    patch_data = {'lab':my_lab,
                  'award': my_award,
                  'contributing_labs':[FDlab,]}
    # if there are other contributing labs, add them to the cont_labs
    if a_file.get('contributing_labs'):
        for cont_lab in a_file['contributing_labs']:
            if cont_lab['uuid'] not in [my_lab, FDlab]:
                patch_data['contributing_labs'].append(cont_lab['uuid'])
    ff_utils.patch_metadata(patch_data,obj_id=a_file['uuid'], key=my_key)

In [ ]:
# find all runs from 4dn lab
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
q = "/search/?type=WorkflowRunAwsem&lab.display_title=4DN%20DCIC%20Lab%2C%20HMS"
all_awsem_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_awsem_resp))

In [ ]:
files_with_cont_and_lab_same = []
# add lab/award/contlab to wfrs
counter = 0
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter)
    
    my_lab = ''
    my_award = ''
    my_cont_lab = ''
    patch_data = {}
    
    input_files = a_resp['input_files']
    run_on_expected = False
    for an_input in input_files:
        # if this wfr ran on processed files
        input_id = an_input['value']['@id']
        if input_id.startswith('/files-processed/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key=my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
        # if this wfr ran on raw files
        elif input_id.startswith('/files-fastq/') or input_id.startswith('/files-microscopy/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key = my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
    # I normally exclude files reference since they are part of many runs, and often belong to dcic
    # in a second round, if the file type is not in one if the expected ones, check what it is 
    # this case often convers md5 runs on reference files or other file types (eg filecalibration)
    if not run_on_expected:
        print('skipping wfr on', input_id)
        continue

            
            
    # if file is not from FDN(user submitted pc), add it to cont labs
    if my_lab is not FDlab:
        if FDlab not in my_cont_lab:
            my_cont_lab.append(FDlab)  

    if my_lab in my_cont_lab:
        my_cont_lab.remove(my_lab)
        files_with_cont_and_lab_same.append(input_id)
        
    patch_data = {'lab':my_lab,
                  'award': my_award}
    if my_cont_lab:
        patch_data['contributing_labs'] = my_cont_lab
        
    ff_utils.patch_metadata(patch_data,obj_id=a_resp['@id'], key=my_key)
